In [1]:
# Instalação das Bibliotecas
!pip install selenium
!pip install webdriver_manager


In [6]:
# Importação das Bibliotecas
import requests
import pandas as pd
import time
import re
import os
import shutil
import unicodedata
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
# Levantamento, Identificação e baixa da lista dos processos
# ================= CONFIGURAÇÕES =================
API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
URL_API = "https://api-publica.datajud.cnj.jus.br/api_publica_tjal/_search"
META_PROCESSOS = 1000 

def criar_query(cursor_paginacao=None):
    """Monta o payload de busca para o ElasticSearch (DataJud)."""
    body = {
        "size": 100, # Traz 100 processos por página
        "query": {
            "bool": {
                "must": [
                    { "match": { "tribunal": "TJAL" } },
                    { "match": { "grau": "G1" } },
                    { "match": { "assuntos.nome": "Consumidor" } },
                    { "range": { "movimentos.dataHora": { "gte": "2020-01-01T00:00:00" } } },
                    { "terms": { "movimentos.codigo": [219, 220, 221] } } # Códigos de Sentença
                ],
                "must_not": [ # Exclui Juizados e Turmas Recursais
                    { "match_phrase": { "orgaoJulgador.nome": "JUIZADO" } },
                    { "match_phrase": { "orgaoJulgador.nome": "TURMA" } },
                    { "match_phrase": { "orgaoJulgador.nome": "RELATOR" } }
                ]
            }
        },
        "sort": [{"@timestamp": "desc"}],
        "_source": ["numeroProcesso", "movimentos", "assuntos", "orgaoJulgador"]
    }
    
    if cursor_paginacao:
        body["search_after"] = cursor_paginacao
        
    return body

def formatar_nup(nup):
    """Aplica a máscara CNJ: NNNNNNN-DD.AAAA.J.TR.OOOO"""
    s = str(nup).strip().zfill(20)
    return f"{s[:7]}-{s[7:9]}.{s[9:13]}.{s[13:14]}.{s[14:16]}.{s[16:20]}"

def main():
    headers = {"Authorization": f"APIKey {API_KEY}"}
    dados_coletados = []
    cursor = None # Ponteiro para a próxima página de resultados

    print(f"--- Iniciando Coleta: Meta {META_PROCESSOS} processos ---")

    while len(dados_coletados) < META_PROCESSOS:
        try:
            # Faz a requisição (o parametro json= lida com a conversão e headers automaticamente)
            resp = requests.post(URL_API, headers=headers, json=criar_query(cursor), timeout=20)
            
            if resp.status_code != 200:
                print(f"⚠️ Erro API: {resp.status_code} - Tentando novamente em 5s...")
                time.sleep(5)
                continue

            hits = resp.json().get('hits', {}).get('hits', [])
            if not hits:
                print("⏹️ Fim dos resultados disponíveis na API.")
                break

            for hit in hits:
                src = hit.get('_source', {})
                orgao = src.get('orgaoJulgador', {}).get('nome', '').upper()

                # Filtro de segurança (caso o ElasticSearch deixe passar algo)
                if "JUIZADO" in orgao or "TURMA" in orgao:
                    continue

                # Extração Simplificada
                nup_raw = src.get('numeroProcesso')
                
                # Pega a lista de assuntos concatenada
                lista_assuntos = [a['nome'] for a in src.get('assuntos', []) if 'nome' in a]
                texto_assuntos = "; ".join(lista_assuntos)

                # Busca o tipo de sentença (Decisão mais recente dentre os códigos alvo)
                codigos_sentenca = [219, 220, 221]
                movs = sorted(src.get('movimentos', []), key=lambda x: x.get('dataHora', ''), reverse=True)
                decisao = next((m['nome'] for m in movs if m.get('codigo') in codigos_sentenca), "Sentença Genérica")

                dados_coletados.append({
                    "NUP": nup_raw,
                    "NUP_Formatado": formatar_nup(nup_raw),
                    "Unidade": orgao,
                    "Assuntos": texto_assuntos,
                    "Decisao": decisao
                })

            # Atualiza o cursor para a próxima página
            cursor = hits[-1]['sort']
            print(f"Processos coletados: {len(dados_coletados)}")
            
            if len(dados_coletados) >= META_PROCESSOS:
                break
                
        except Exception as e:
            print(f"❌ Erro crítico: {e}")
            break

    # Salva o resultado
    if dados_coletados:
        df = pd.DataFrame(dados_coletados[:META_PROCESSOS]) # Garante o corte exato
        df.to_csv("listaaaa_processos_tjal.csv", index=False, sep=';', encoding='utf-8-sig')
        print(f"\n✅ Arquivo 'lista_processos_tjal.csv' gerado com {len(df)} registros.")
    else:
        print("\n⚠️ Nenhum dado encontrado.")

if __name__ == "__main__":
    main()

--- Iniciando Coleta: Meta 1000 processos ---
Processos coletados: 100
Processos coletados: 200
Processos coletados: 300
Processos coletados: 400
Processos coletados: 500
Processos coletados: 600
Processos coletados: 700
Processos coletados: 800
Processos coletados: 900
Processos coletados: 1000

✅ Arquivo 'lista_processos_tjal.csv' gerado com 1000 registros.


In [12]:
# Baixar os arquivos em PDF dos processos julgados

# ================= CONFIGURAÇÕES =================
ARQUIVO_ENTRADA = "listaaaa_processos_tjal.csv"

## O ENDEREÇO DA PASTA DESTINO PRECISA SER AJUSTADO SEMPRE DE ACORDO COM A PASTA DESEJADA DE BAIXA DOS ARQUIVOS 
PASTA_DESTINO = r"C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos" 
URL_ESAJ = "https://www2.tjal.jus.br/cpopg/open.do"

if not os.path.exists(PASTA_DESTINO):
    os.makedirs(PASTA_DESTINO)

def configurar_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    
    prefs = {
        "download.default_directory": PASTA_DESTINO,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": False, 
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def formatar_nup(nup):
    clean = re.sub(r'\D', '', str(nup)).zfill(20)
    return {
        "num_unificado": f"{clean[0:7]}-{clean[7:9]}.{clean[9:13]}",
        "foro": clean[16:20]
    }

def expandir_movimentacoes(driver):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            link_mov = driver.find_element(By.ID, "linkMovimentacoes")
            if link_mov.is_displayed():
                driver.execute_script("arguments[0].click();", link_mov)
                time.sleep(1)
                return
        except: pass

        links = driver.find_elements(By.TAG_NAME, "a")
        for link in links:
            if "Mais" in link.text and link.is_displayed():
                try:
                    driver.execute_script("arguments[0].click();", link)
                    time.sleep(0.5)
                except: pass
    except: pass

def esperar_novo_arquivo_e_renomear(pasta, arquivos_antes, nup_formatado, timeout=60):
    print("   > Aguardando novo arquivo...", end="\r")
    fim = time.time() + timeout
    
    while time.time() < fim:
        try:
            arquivos_agora = set(os.listdir(pasta))
            novos = arquivos_agora - arquivos_antes
            novos_pdfs = [f for f in novos if f.endswith(".pdf")]
            
            if novos_pdfs:
                nome_arquivo_novo = novos_pdfs[0]
                caminho_origem = os.path.join(pasta, nome_arquivo_novo)
                
                time.sleep(1) 
                
                caminho_final = os.path.join(pasta, f"{nup_formatado}.pdf")
                
                if os.path.exists(caminho_final):
                    try: os.remove(caminho_final)
                    except: pass
                
                tentativas = 0
                while tentativas < 5:
                    try:
                        os.rename(caminho_origem, caminho_final)
                        return True
                    except PermissionError:
                        time.sleep(1)
                        tentativas += 1
                    except Exception:
                        return False
        except: pass
                    
        time.sleep(1)
        
    return False

def tentar_clicar_botoes(driver):
    seletores = [
        (By.ID, "download"), 
        (By.CSS_SELECTOR, "button[title='Baixar']"),
        (By.CSS_SELECTOR, "button[title='Download']"),
        (By.ID, "botaoSalvar"),
        (By.CSS_SELECTOR, ".toolbarButton.download"),
        (By.ID, "secondaryDownload")
    ]
    
    for by, selector in seletores:
        try:
            botao = driver.find_element(by, selector)
            if botao.is_displayed():
                driver.execute_script("arguments[0].click();", botao)
                return True
        except: continue
    return False

def clicar_botao_baixar_popup(driver, wait):
    try:
        if tentar_clicar_botoes(driver): return True
            
        frames = driver.find_elements(By.TAG_NAME, "iframe")
        for frame in frames:
            try:
                driver.switch_to.frame(frame)
                if tentar_clicar_botoes(driver):
                    driver.switch_to.default_content()
                    return True
                
                subframes = driver.find_elements(By.TAG_NAME, "iframe")
                for subframe in subframes:
                    driver.switch_to.frame(subframe)
                    if tentar_clicar_botoes(driver):
                        driver.switch_to.default_content()
                        return True
                    driver.switch_to.parent_frame()

                driver.switch_to.default_content()
            except:
                driver.switch_to.default_content()
    except: pass
    return False

def processar_sentenca(driver, wait, nup_formatado):
    """
    Retorna uma TUPLA: (status, texto_da_movimentacao)
    """
    expandir_movimentacoes(driver)
    gatilhos = ["julgado improcedente", "julgado procedente", "sentença", "extinguindo", "mérito"]
    
    try:
        wait.until(EC.presence_of_element_located((By.ID, "tabelaTodasMovimentacoes")))
        linhas = driver.find_elements(By.CSS_SELECTOR, "#tabelaTodasMovimentacoes tr")
        janela_principal = driver.current_window_handle
        
        for linha in linhas:
            texto_linha = linha.text
            texto_lower = texto_linha.lower()
            
            if any(g in texto_lower for g in gatilhos):
                # --- CAPTURA O TEXTO AQUI ---
                # Remove quebras de linha para ficar numa linha só no Excel
                teor_movimentacao = texto_linha.replace("\n", " || ").strip()
                
                try:
                    links = linha.find_elements(By.TAG_NAME, "a")
                    for link in links:
                        if link.is_displayed():
                            
                            arquivos_antes = set(os.listdir(PASTA_DESTINO))

                            driver.execute_script("arguments[0].click();", link)
                            wait.until(EC.number_of_windows_to_be(2))
                            
                            janelas = driver.window_handles
                            driver.switch_to.window(janelas[-1])
                            time.sleep(3) 
                            
                            clicou = clicar_botao_baixar_popup(driver, WebDriverWait(driver, 5))
                            
                            status = "Erro Clique"
                            if clicou:
                                if esperar_novo_arquivo_e_renomear(PASTA_DESTINO, arquivos_antes, nup_formatado):
                                    status = "Sucesso"
                                else:
                                    status = "Erro: Arquivo não apareceu"
                            
                            driver.close()
                            driver.switch_to.window(janela_principal)
                            
                            # Retorna o Status E o Texto capturado
                            return status, teor_movimentacao

                except Exception as e:
                    print(f"   > Erro linha: {e}")
                    if len(driver.window_handles) > 1:
                        driver.close()
                        driver.switch_to.window(janela_principal)
                    continue

    except Exception as e:
        return f"Erro Geral: {str(e)}", ""
        
    return "Documento não encontrado", ""

def main():
    if not os.path.exists(ARQUIVO_ENTRADA):
        return print(f"❌ Arquivo '{ARQUIVO_ENTRADA}' não encontrado.")

    print(f"--- Iniciando Robô (Download PDF + Captura de Texto) ---")
    print(f"📂 Pasta: {PASTA_DESTINO}")
    
    df = pd.read_csv(ARQUIVO_ENTRADA, sep=';', encoding='utf-8-sig')
    
    # === TESTE COM 10 ===
    #df = df.head(10)
    # ====================

    if 'Status_Download' not in df.columns: df['Status_Download'] = None
    if 'Magistrado_Scraping' not in df.columns: df['Magistrado_Scraping'] = None
    # Nova coluna para o texto da tabela
    if 'Teor_Movimentacao' not in df.columns: df['Teor_Movimentacao'] = None

    driver = configurar_driver()
    wait = WebDriverWait(driver, 20)

    try:
        for index, row in df.iterrows():
            nup = row.get('NUP_Formatado', row.get('Processo', ''))
            print(f"\n[{index+1}/{len(df)}] Processando: {nup}")

            try:
                driver.get(URL_ESAJ)
                dados = formatar_nup(nup)

                driver.execute_script("arguments[0].value = arguments[1];", wait.until(EC.presence_of_element_located((By.ID, "numeroDigitoAnoUnificado"))), dados['num_unificado'])
                driver.execute_script("arguments[0].value = arguments[1];", driver.find_element(By.ID, "foroNumeroUnificado"), dados['foro'])
                driver.find_element(By.ID, "botaoConsultarProcessos").click()

                wait.until(lambda d: d.find_elements(By.CLASS_NAME, "secaoFormBody") or d.find_elements(By.ID, "mensagemRetorno") or d.find_elements(By.ID, "senhaProcesso") or d.find_elements(By.ID, "processoSelecionado"))

                if driver.find_elements(By.ID, "senhaProcesso") and driver.find_elements(By.ID, "senhaProcesso")[0].is_displayed():
                    print("   > 🔒 Segredo")
                    continue
                if driver.find_elements(By.ID, "mensagemRetorno") and "não existem" in driver.find_elements(By.ID, "mensagemRetorno")[0].text.lower():
                    print("   > ❌ Não encontrado")
                    continue
                if driver.find_elements(By.ID, "processoSelecionado"):
                    driver.find_elements(By.ID, "processoSelecionado")[0].click()
                    driver.find_element(By.ID, "botaoEnviar").click()
                    time.sleep(1)

                try:
                    juiz = driver.find_element(By.ID, "juizProcesso").text.strip()
                    df.at[index, 'Magistrado_Scraping'] = juiz
                except: pass

                # Chama a função que retorna STATUS e TEXTO
                status, texto_mov = processar_sentenca(driver, wait, nup)
                
                if status == "Sucesso":
                    print(f"   > 💾 PDF Salvo!")
                    if texto_mov:
                        print(f"   > 📜 Texto: {texto_mov[:50]}...")
                else:
                    print(f"   > ⚠️ {status}")

                df.at[index, 'Status_Download'] = status
                df.at[index, 'Teor_Movimentacao'] = texto_mov

            except Exception as e:
                print(f"   > ⚠️ Erro crítico: {e}")
                
    except KeyboardInterrupt:
        print("\n⏹️ Parado.")
    finally:
        driver.quit()
        df.to_csv(os.path.join(PASTA_DESTINO, "log_downloads.csv"), index=False, sep=';', encoding='utf-8-sig')
        print(f"\n✅ Finalizado! Log salvo com textos.")

if __name__ == "__main__":
    main()

--- Iniciando Robô (Download PDF + Captura de Texto) ---
📂 Pasta: C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos

[1/1000] Processando: 0700353-06.2021.8.02.0204
   > 💾 PDF Salvo!vo arquivo...
   > 📜 Texto: 13/06/2024 Julgado procedente o pedido || III - DI...

[2/1000] Processando: 0700174-16.2021.8.02.0061
   > 💾 PDF Salvo!vo arquivo...
   > 📜 Texto: 08/08/2024 Julgado improcedente o pedido || III DI...

[3/1000] Processando: 0700159-72.2020.8.02.0064
   > Erro linha: Message: 

   > 💾 PDF Salvo!vo arquivo...
   > 📜 Texto: 07/02/2023 Decisão Proferida || DECISÃO Trata-se d...

[4/1000] Processando: 0700653-12.2021.8.02.0060
   > Erro linha: Message: 

   > 💾 PDF Salvo!vo arquivo...
   > 📜 Texto: 23/10/2025 Decisão Proferida || 1. Ab initio, PROM...

[5/1000] Processando: 0712422-23.2021.8.02.0058
   > 💾 PDF Salvo!vo arquivo...
   > 📜 Texto: 09/09/2025 Extinta a execução ou o cumprimento da ...

[6/1000] Processando: 0

In [13]:
# CONFERÊNCIA DO TOTAL DE ARQUIVOS BAIXADOS
# ================= CONFIGURAÇÕES =================
## O ENDEREÇO DA PASTA DESTINO PRECISA SER AJUSTADO SEMPRE DE ACORDO COM A PASTA DESEJADA DE BAIXA DOS ARQUIVOS 
# O mesmo caminho onde você salvou os arquivos
PASTA_DESTINO = r"C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos" 
NOME_ARQUIVO = "log_downloads.csv"

# Cria o caminho completo
caminho_completo = os.path.join(PASTA_DESTINO, NOME_ARQUIVO)

# ================= CARREGAMENTO =================
df_log = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    
print("✅ Arquivo carregado com sucesso!")
print(f"Total de registros: {len(df_log)}")
    


✅ Arquivo carregado com sucesso!
Total de registros: 1000


In [14]:
    # Mostra as primeiras 5 linhas para conferência
print("\n--- Amostra dos Dados ---")
df_log.head()


--- Amostra dos Dados ---


,NUP,NUP_Formatado,Unidade,Assuntos,Decisao,Status_Download,Magistrado_Scraping,Teor_Movimentacao
0,7003530620218020204,0700353-06.2021.8.02.0204,FORO DE BATALHA - VARA DO ÚNICO OFÍCIO DE BATALHA,DIREITO DO CONSUMIDOR,Procedência,Sucesso,Diego Cadore Pedroso,13/06/2024 Julgado procedente o pedido || III ...
1,7001741620218020061,0700174-16.2021.8.02.0061,FORO DE RIO LARGO - 1ª VARA DE RIO LARGO /CÍVE...,DIREITO DO CONSUMIDOR,Improcedência,Sucesso,Larrissa Gabriella Lins Victor Lacerda,08/08/2024 Julgado improcedente o pedido || II...
2,7001597220208020064,0700159-72.2020.8.02.0064,FORO DE TAQUARANA - VARA DO ÚNICO OFÍCIO DE TA...,DIREITO DO CONSUMIDOR,Procedência em Parte,Sucesso,Eduardo Ligiéro Rocha,07/02/2023 Decisão Proferida || DECISÃO Trata-...
3,7006531220218020060,0700653-12.2021.8.02.0060,FORO DE FEIRA GRANDE - VARA DO ÚNICO OFÍCIO DE...,DIREITO DO CONSUMIDOR,Procedência em Parte,Sucesso,Darlan Soares Souza,"23/10/2025 Decisão Proferida || 1. Ab initio, ..."
4,7124222320218020058,0712422-23.2021.8.02.0058,FORO DE ARAPIRACA - 3ª VARA DE ARAPIRACA / CÍV...,DIREITO DO CONSUMIDOR,Procedência em Parte,Sucesso,Carlos Bruno de Oliveira Ramos,09/09/2025 Extinta a execução ou o cumprimento...


In [15]:
   # Mostra informações das colunas (para ver se tem nulos)
print("\n--- Estrutura do Arquivo ---")
df_log.info()


--- Estrutura do Arquivo ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUP                  1000 non-null   int64 
 1   NUP_Formatado        1000 non-null   object
 2   Unidade              1000 non-null   object
 3   Assuntos             996 non-null    object
 4   Decisao              1000 non-null   object
 5   Status_Download      954 non-null    object
 6   Magistrado_Scraping  951 non-null    object
 7   Teor_Movimentacao    952 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


In [16]:
df_limpo = df_log.dropna()
colunas_finais = ['NUP_Formatado', 'Decisao', 'Magistrado_Scraping']
df_final = df_limpo[colunas_finais].copy()
df_final = df_final.reset_index(drop=True)

print(f"Linhas antes: {len(df_log)}")
print(f"Linhas depois: {len(df_limpo)}")
df_final.info()
df_final.head()

Linhas antes: 1000
Linhas depois: 945
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUP_Formatado        945 non-null    object
 1   Decisao              945 non-null    object
 2   Magistrado_Scraping  945 non-null    object
dtypes: object(3)
memory usage: 22.3+ KB


,NUP_Formatado,Decisao,Magistrado_Scraping
0,0700353-06.2021.8.02.0204,Procedência,Diego Cadore Pedroso
1,0700174-16.2021.8.02.0061,Improcedência,Larrissa Gabriella Lins Victor Lacerda
2,0700159-72.2020.8.02.0064,Procedência em Parte,Eduardo Ligiéro Rocha
3,0700653-12.2021.8.02.0060,Procedência em Parte,Darlan Soares Souza
4,0712422-23.2021.8.02.0058,Procedência em Parte,Carlos Bruno de Oliveira Ramos


In [17]:
print("--- Tipos de Decisão encontrados ---")
print(df_final['Decisao'].unique())

--- Tipos de Decisão encontrados ---
['Procedência' 'Improcedência' 'Procedência em Parte']


In [18]:
mapa_decisao = {
    'Improcedência': 0,        # Negativo
    'Procedência em Parte': 1, # Neutro/Misto
    'Procedência': 2           # Positivo
}

df_final['Target'] = df_final['Decisao'].map(mapa_decisao)
df_final['Target'] = df_final['Target'].astype(int)

print("--- Distribuição das Classes ---")
print(df_final['Target'].value_counts().sort_index())
print("\n--- Visualização Final ---")
print(df_final.head())

--- Distribuição das Classes ---
Target
0    368
1    348
2    229
Name: count, dtype: int64

--- Visualização Final ---
               NUP_Formatado               Decisao  \
0  0700353-06.2021.8.02.0204           Procedência   
1  0700174-16.2021.8.02.0061         Improcedência   
2  0700159-72.2020.8.02.0064  Procedência em Parte   
3  0700653-12.2021.8.02.0060  Procedência em Parte   
4  0712422-23.2021.8.02.0058  Procedência em Parte   

                      Magistrado_Scraping  Target  
0                    Diego Cadore Pedroso       2  
1  Larrissa Gabriella Lins Victor Lacerda       0  
2                   Eduardo Ligiéro Rocha       1  
3                     Darlan Soares Souza       1  
4          Carlos Bruno de Oliveira Ramos       1  


In [19]:
# Função para remover acentos (JOSÉ -> JOSE)
def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    # Normaliza unicode (separa o acento da letra)
    nfkd = unicodedata.normalize('NFKD', texto)
    # Filtra apenas caracteres não-acento e converte para maiúsculo
    texto_sem_acento = "".join([c for c in nfkd if not unicodedata.combining(c)])
    return texto_sem_acento.upper().strip()

# ================= 1. PREPARAÇÃO DOS DADOS =================

# Cria uma coluna temporária só com o PRIMEIRO NOME LIMPO
# Ex: "André Luis..." -> "ANDRE"
df_final['Primeiro_Nome_Limpo'] = df_final['Magistrado_Scraping'].apply(
    lambda x: normalizar_texto(str(x).split()[0])
)

# Pega a lista de nomes únicos limpos
nomes_unicos = df_final['Primeiro_Nome_Limpo'].unique()
print(f"Total de nomes únicos para consultar: {len(nomes_unicos)}")

# ================= 2. CONSULTA API IBGE (NORMALIZADA) =================
mapa_genero = {}

# Processa em lotes
tamanho_lote = 30 # Aumentei um pouco o lote
for i in range(0, len(nomes_unicos), tamanho_lote):
    lote = nomes_unicos[i:i+tamanho_lote]
    url_nomes = ",".join(lote) # Envia "JOSE,ANDRE,MARIA"
    
    try:
        url = f"https://servicodados.ibge.gov.br/api/v2/censos/nomes/{url_nomes}"
        response = requests.get(url)
        
        if response.status_code == 200:
            dados = response.json()
            for item in dados:
                # O IBGE pode devolver com ou sem acento, então normalizamos a chave também
                nome_ibge = normalizar_texto(item['nome'])
                sexo = item['res'][0]['sexo'] 
                
                # Mapeia: M=0, F=1
                mapa_genero[nome_ibge] = 0 if sexo == 'M' else 1
        
        print(f"Lote {i} processado...")
        time.sleep(0.5) 
        
    except Exception as e:
        print(f"Erro de conexão: {e}")

# ================= 3. APLICAÇÃO E FALLBACK (REGRA DO 'A') =================

def resolver_genero_final(primeiro_nome_limpo):
    # 1. Tenta achar no IBGE
    if primeiro_nome_limpo in mapa_genero:
        return mapa_genero[primeiro_nome_limpo]
    
    # 2. Se não achou (ex: Larrissa, Bruce), aplica a "Regra da Vogal A"
    # Se terminar em A -> Feminino (1), senão Masculino (0)
    # Essa regra salva os nomes estrangeiros ou com grafia rara (Larrissa, Allysson)
    if primeiro_nome_limpo.endswith('A'):
        return 1
    else:
        return 0

# Aplica a lógica usando a coluna LIMPA
df_final['Genero'] = df_final['Primeiro_Nome_Limpo'].map(resolver_genero_final)

# Limpeza: remove a coluna auxiliar
df_final = df_final.drop(columns=['Primeiro_Nome_Limpo'])

# ================= 4. RELATÓRIO =================
print("\n--- Resultado Final ---")
print(df_final['Genero'].value_counts().sort_index())
print("Legenda: 0=Masc | 1=Fem")

print("\n--- Amostra ---")
print(df_final[['Magistrado_Scraping', 'Genero']].head(10))

Total de nomes únicos para consultar: 65
Lote 0 processado...
Lote 30 processado...
Lote 60 processado...

--- Resultado Final ---
Genero
0    733
1    212
Name: count, dtype: int64
Legenda: 0=Masc | 1=Fem

--- Amostra ---
                      Magistrado_Scraping  Genero
0                    Diego Cadore Pedroso       0
1  Larrissa Gabriella Lins Victor Lacerda       1
2                   Eduardo Ligiéro Rocha       0
3                     Darlan Soares Souza       0
4          Carlos Bruno de Oliveira Ramos       0
5          Carlos Bruno de Oliveira Ramos       0
6          Carlos Bruno de Oliveira Ramos       0
7          Carlos Bruno de Oliveira Ramos       0
8          Luciana Josué Raposo Lima Dias       1
9          Carlos Bruno de Oliveira Ramos       0


In [20]:
# 1. Separar os dados por gênero usando o DF correto (df_final) e a coluna 'Genero'
df_genero_0 = df_final[df_final['Genero'] == 0]
df_genero_1 = df_final[df_final['Genero'] == 1]

# 2. Selecionar aleatoriamente 317 observações do gênero 0
# Usamos random_state=42 para garantir que o sorteio seja sempre o mesmo
if len(df_genero_0) >= 317:
    df_genero_0_sample = df_genero_0.sample(n=317, random_state=42)
else:
    print(f"Aviso: Existem apenas {len(df_genero_0)} homens. Todos foram selecionados.")
    df_genero_0_sample = df_genero_0

# 3. Concatenar a amostra sorteada do gênero 0 com TODOS do gênero 1
df_amostra_final = pd.concat([df_genero_0_sample, df_genero_1])

# 4. Embaralhar o resultado final (opcional, para misturar os gêneros no arquivo)
df_amostra_final = df_amostra_final.sample(frac=1, random_state=42).reset_index(drop=True)

# 5. Conferindo o resultado
print("--- Contagem Final ---")
print(df_amostra_final['Genero'].value_counts())
df_amostra_final.head()

--- Contagem Final ---
Genero
0    317
1    212
Name: count, dtype: int64


,NUP_Formatado,Decisao,Magistrado_Scraping,Target,Genero
0,0701876-06.2021.8.02.0058,Procedência,José Miranda Santos Junior,2,0
1,0706300-28.2020.8.02.0058,Improcedência,Luciana Josué Raposo Lima Dias,0,1
2,0700082-34.2021.8.02.0030,Improcedência,Bruce Lee Simões Pimentel,0,0
3,0700802-55.2021.8.02.0012,Improcedência,Natália Cerqueira de Castro,0,1
4,0700226-42.2021.8.02.0051,Procedência em Parte,Larrissa Gabriella Lins Victor Lacerda,1,1


In [21]:
# 1. Define o caminho da pasta e o nome do arquivo
## O ENDEREÇO DA PASTA DESTINO PRECISA SER AJUSTADO SEMPRE DE ACORDO COM A PASTA DESEJADA DE BAIXA DOS ARQUIVOS 
# O mesmo caminho onde você salvou os arquivos
# O 'r' antes das aspas é essencial para caminhos do Windows
caminho_pasta = r'C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos'
nome_arquivo = 'amostra_processos_final.csv'

# 2. Cria o caminho completo (junta pasta + arquivo de forma segura)
caminho_completo = os.path.join(caminho_pasta, nome_arquivo)

# (Opcional) Cria a pasta automaticamente se ela ainda não existir
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)

# 3. Salva o arquivo
df_amostra_final.to_csv(caminho_completo, index=False, sep=',', encoding='utf-8-sig')

print(f"Arquivo salvo com sucesso em: {caminho_completo}")

Arquivo salvo com sucesso em: C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos\amostra_processos_final.csv


In [24]:
# 1. Configurações
PASTA_BASE = r'C:\Iris\Mestrado_IDP\05_DEEP Learning e Processamento de Linguagem Natural\Trabalho final Deep Learning\Processos'
caminho_csv = os.path.join(PASTA_BASE, 'amostra_processos_final.csv')

# 2. Carrega o CSV
df = pd.read_csv(caminho_csv)

print("--- Rastreando arquivo ausente ---\n")

encontrou_erro = False

for index, row in df.iterrows():
    nup = row['NUP_Formatado']
    nome_arquivo = f"{nup}.pdf"
    caminho_completo = os.path.join(PASTA_BASE, nome_arquivo)
    
    # Se NÃO existir, imprime na hora
    if not os.path.exists(caminho_completo):
        print(f"❌ FALTOU ESTE: {nome_arquivo}")
        encontrou_erro = True

if not encontrou_erro:
    print("Estranho... agora o script diz que encontrou todos.")

--- Rastreando arquivo ausente ---

Estranho... agora o script diz que encontrou todos.


In [23]:
df

,NUP_Formatado,Decisao,Magistrado_Scraping,Target,Genero
0,0701876-06.2021.8.02.0058,Procedência,José Miranda Santos Junior,2,0
1,0706300-28.2020.8.02.0058,Improcedência,Luciana Josué Raposo Lima Dias,0,1
2,0700082-34.2021.8.02.0030,Improcedência,Bruce Lee Simões Pimentel,0,0
3,0700802-55.2021.8.02.0012,Improcedência,Natália Cerqueira de Castro,0,1
4,0700226-42.2021.8.02.0051,Procedência em Parte,Larrissa Gabriella Lins Victor Lacerda,1,1
...,...,...,...,...,...
524,0700135-61.2020.8.02.0026,Procedência em Parte,Rogério Santos Alencar,1,0
525,0736932-77.2021.8.02.0001,Procedência em Parte,Sérgio Wanderley Persiano,1,0
526,0701101-61.2020.8.02.0046,Procedência,Wilians Alencar Coelho Junior,2,0
527,0708898-18.2021.8.02.0058,Procedência em Parte,Luciana Josué Raposo Lima Dias,1,1
